In [1]:
import numpy as np 
import pandas as pd

In [2]:
%cd ../../

d:\dev\project\Football-Match-Prediction


In [3]:
df = pd.read_csv('data/raw/all_teams_data.csv')
pd.options.display.max_columns = None
df2 = df[df['Comp'] == 'La Liga']
df2.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss_x,Attendance,Captain,Formation,Opp Formation,Referee,Notes,Standard__Gls,Standard__Sh,Standard__SoT,Standard__SoT%,Standard__G/Sh,Standard__G/SoT,Standard__Dist,Standard__FK,Standard__PK,Standard__PKatt,Expected__xG,Expected__npxG,Expected__npxG/Sh,Expected__G-xG,Expected__np:G-xG,Performance__SoTA,Performance__GA,Performance__Saves,Performance__Save%,Performance__CS,Performance__PSxG,Performance__PSxG+/-,Penalty Kicks__PKatt,Penalty Kicks__PKA,Penalty Kicks__PKsv,Penalty Kicks__PKm,Launched__Cmp,Launched__Att,Launched__Cmp%,Passes__Att (GK),Passes__Thr,Passes__Launch%,Passes__AvgLen,Goal Kicks__Att,Goal Kicks__Launch%,Goal Kicks__AvgLen,Crosses__Opp,Crosses__Stp,Crosses__Stp%,Sweeper__#OPA,Sweeper__AvgDist,Total__Cmp,Total__Att,Total__Cmp%,Total__TotDist,Total__PrgDist,Short__Cmp,Short__Att,Short__Cmp%,Medium__Cmp,Medium__Att,Medium__Cmp%,Long__Cmp,Long__Att,Long__Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Att,Pass Types__Live,Pass Types__Dead,Pass Types__FK,Pass Types__TB,Pass Types__Sw,Pass Types__Crs,Pass Types__TI,Pass Types__CK,Corner Kicks__In,Corner Kicks__Out,Corner Kicks__Str,Outcomes__Cmp,Outcomes__Off,Outcomes__Blocks,SCA Types__SCA,SCA Types__PassLive,SCA Types__PassDead,SCA Types__TO,SCA Types__Sh,SCA Types__Fld,SCA Types__Def,GCA Types__GCA,GCA Types__PassLive,GCA Types__PassDead,GCA Types__TO,GCA Types__Sh,GCA Types__Fld,GCA Types__Def,Tackles__Tkl,Tackles__TklW,Tackles__Def 3rd,Tackles__Mid 3rd,Tackles__Att 3rd,Challenges__Tkl,Challenges__Att,Challenges__Tkl%,Challenges__Lost,Blocks__Blocks,Blocks__Sh,Blocks__Pass,Int,Tkl+Int,Clr,Err,Poss_y,Touches__Touches,Touches__Def Pen,Touches__Def 3rd,Touches__Mid 3rd,Touches__Att 3rd,Touches__Att Pen,Touches__Live,Take-Ons__Att,Take-Ons__Succ,Take-Ons__Succ%,Take-Ons__Tkld,Take-Ons__Tkld%,Carries__Carries,Carries__TotDist,Carries__PrgDist,Carries__PrgC,Carries__1/3,Carries__CPA,Carries__Mis,Carries__Dis,Receiving__Rec,Receiving__PrgR,Performance__CrdY,Performance__CrdR,Performance__2CrdY,Performance__Fls,Performance__Fld,Performance__Off,Performance__Crs,Performance__Int,Performance__TklW,Performance__PKwon,Performance__PKcon,Performance__OG,Performance__Recov,Aerial Duels__Won,Aerial Duels__Lost,Aerial Duels__Won%,Season,Team
0,2017-08-18,20:15,La Liga,Matchweek 1,Fri,Away,L,0,1,Leganés,1.1,1.3,47.0,9231.0,Manu García,4-4-2,4-2-3-1,José Munuera,NaN,0,8.0,4.0,50.0,0.0,0.0,22.8,0.0,0,1,1.1,0.4,0.04,-1.1,-0.4,3.0,1.0,2.0,66.7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,21.0,23.8,30.0,4.0,46.7,33.0,7.0,100.0,57.3,11.0,1.0,9.1,2.0,14.3,252.0,385.0,65.5,4587.0,1900.0,119.0,145.0,82.1,98.0,138.0,71.0,33.0,82.0,40.2,0,0.3,0.4,7.0,19.0,2.0,0.0,18.0,385.0,330.0,55.0,19.0,1.0,5.0,10.0,25.0,2.0,1.0,0.0,1.0,252.0,0.0,10.0,15.0,7.0,3.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,13.0,10.0,9.0,0.0,8.0,15.0,53.3,7.0,14.0,5.0,9.0,14.0,33,17.0,1.0,47.0,503.0,47.0,200.0,222.0,89.0,10.0,502.0,18.0,8.0,44.4,10.0,55.6,235.0,1359.0,652.0,10.0,8.0,4.0,19.0,13.0,250.0,18.0,1,0,0,17.0,17.0,0.0,10.0,14.0,13.0,1.0,0.0,0.0,60.0,16.0,30.0,34.8,2017,Alaves
1,2017-08-26,18:15,La Liga,Matchweek 2,Sat,Home,L,0,2,Barcelona,0.9,2.5,27.0,19356.0,Manu García,4-5-1,4-3-3,Carlos del Cerro,NaN,0,11.0,2.0,18.2,0.0,0.0,21.7,2.0,0,0,0.9,0.9,0.09,-0.9,-0.9,11.0,2.0,8.0,81.8,0.0,2.3,0.3,1.0,0.0,1.0,0.0,10.0,17.0,58.8,26.0,4.0,53.8,38.9,5.0,60.0,48.6,13.0,0.0,0.0,0.0,7.7,202.0,281.0,71.9,3959.0,1914.0,85.0,99.0,85.9,73.0,97.0,75.3,34.0,67.0,50.7,0,0.8,0.3,6.0,14.0,1.0,0.0,13.0,281.0,257.0,23.0,7.0,1.0,4.0,5.0,7.0,1.0,1.0,0.0,0.0,202.0,1.0,4.0,19.0,9.0,2.0,2.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,18.0,19.0,4.0,2.0,11.0,32.0,34.4,21.0,14.0,7.0,7.0,15.0,40,30.0,1.0,27.0,396.0,80.0,213.0,140.0,48.0,9.0,396.0,16.0,8.0,50.0,8.0,50.0,258.0,1290.0,662.0,7.0,2.0,4.0,10.0,5.0,199.0,13.0,2,0,0,12.0,9.0,1.0,5.0,15.0,18.0,0.0,1.0,0.0,57.0,8.0,6.0,57.1,2017,Alaves
2,2017-09-10,18:30,La Liga,Matchweek 3,Sun,Away,L,0,1,Celta Vigo,0.5,2.7,34.0,17384.0,Fernando Pacheco,4-

In [4]:
df2.columns.to_list()

['Date',
 'Time',
 'Comp',
 'Round',
 'Day',
 'Venue',
 'Result',
 'GF',
 'GA',
 'Opponent',
 'xG',
 'xGA',
 'Poss_x',
 'Attendance',
 'Captain',
 'Formation',
 'Opp Formation',
 'Referee',
 'Notes',
 'Standard__Gls',
 'Standard__Sh',
 'Standard__SoT',
 'Standard__SoT%',
 'Standard__G/Sh',
 'Standard__G/SoT',
 'Standard__Dist',
 'Standard__FK',
 'Standard__PK',
 'Standard__PKatt',
 'Expected__xG',
 'Expected__npxG',
 'Expected__npxG/Sh',
 'Expected__G-xG',
 'Expected__np:G-xG',
 'Performance__SoTA',
 'Performance__GA',
 'Performance__Saves',
 'Performance__Save%',
 'Performance__CS',
 'Performance__PSxG',
 'Performance__PSxG+/-',
 'Penalty Kicks__PKatt',
 'Penalty Kicks__PKA',
 'Penalty Kicks__PKsv',
 'Penalty Kicks__PKm',
 'Launched__Cmp',
 'Launched__Att',
 'Launched__Cmp%',
 'Passes__Att (GK)',
 'Passes__Thr',
 'Passes__Launch%',
 'Passes__AvgLen',
 'Goal Kicks__Att',
 'Goal Kicks__Launch%',
 'Goal Kicks__AvgLen',
 'Crosses__Opp',
 'Crosses__Stp',
 'Crosses__Stp%',
 'Sweeper__#OPA',

In [5]:
df2.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 5520 entries, 0 to 6629
Data columns (total 168 columns):
 #    Column                Dtype  
---   ------                -----  
 0    Date                  object 
 1    Time                  object 
 2    Comp                  object 
 3    Round                 object 
 4    Day                   object 
 5    Venue                 object 
 6    Result                object 
 7    GF                    object 
 8    GA                    object 
 9    Opponent              object 
 10   xG                    float64
 11   xGA                   float64
 12   Poss_x                float64
 13   Attendance            float64
 14   Captain               object 
 15   Formation             object 
 16   Opp Formation         object 
 17   Referee               object 
 18   Notes                 object 
 19   Standard__Gls         int64  
 20   Standard__Sh          float64
 21   Standard__SoT         float64
 22   Standard__SoT%        float

In [6]:
%cd src

d:\dev\project\Football-Match-Prediction\src


In [7]:
from preprocessing.preprocess import *
df2 = preprocess_categorical_data(df2)
df2

,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss_x,Attendance,Captain,Formation,Opp Formation,Referee,Standard__Gls,Standard__Sh,Standard__SoT,Standard__SoT%,Standard__G/Sh,Standard__G/SoT,Standard__Dist,Standard__FK,Standard__PK,Standard__PKatt,Expected__xG,Expected__npxG,Expected__npxG/Sh,Expected__G-xG,Expected__np:G-xG,Performance__SoTA,Performance__GA,Performance__Saves,Performance__Save%,Performance__CS,Performance__PSxG,Performance__PSxG+/-,Penalty Kicks__PKatt,Penalty Kicks__PKA,Penalty Kicks__PKsv,Penalty Kicks__PKm,Launched__Cmp,Launched__Att,Launched__Cmp%,Passes__Att (GK),Passes__Thr,Passes__Launch%,Passes__AvgLen,Goal Kicks__Att,Goal Kicks__Launch%,Goal Kicks__AvgLen,Crosses__Opp,Crosses__Stp,Crosses__Stp%,Sweeper__#OPA,Sweeper__AvgDist,Total__Cmp,Total__Att,Total__Cmp%,Total__TotDist,Total__PrgDist,Short__Cmp,Short__Att,Short__Cmp%,Medium__Cmp,Medium__Att,Medium__Cmp%,Long__Cmp,Long__Att,Long__Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Att,Pass Types__Live,Pass Types__Dead,Pass Types__FK,Pass Types__TB,Pass Types__Sw,Pass Types__Crs,Pass Types__TI,Pass Types__CK,Corner Kicks__In,Corner Kicks__Out,Corner Kicks__Str,Outcomes__Cmp,Outcomes__Off,Outcomes__Blocks,SCA Types__SCA,SCA Types__PassLive,SCA Types__PassDead,SCA Types__TO,SCA Types__Sh,SCA Types__Fld,SCA Types__Def,GCA Types__GCA,GCA Types__PassLive,GCA Types__PassDead,GCA Types__TO,GCA Types__Sh,GCA Types__Fld,GCA Types__Def,Tackles__Tkl,Tackles__TklW,Tackles__Def 3rd,Tackles__Mid 3rd,Tackles__Att 3rd,Challenges__Tkl,Challenges__Att,Challenges__Tkl%,Challenges__Lost,Blocks__Blocks,Blocks__Sh,Blocks__Pass,Int,Tkl+Int,Clr,Err,Poss_y,Touches__Touches,Touches__Def Pen,Touches__Def 3rd,Touches__Mid 3rd,Touches__Att 3rd,Touches__Att Pen,Touches__Live,Take-Ons__Att,Take-Ons__Succ,Take-Ons__Succ%,Take-Ons__Tkld,Take-Ons__Tkld%,Carries__Carries,Carries__TotDist,Carries__PrgDist,Carries__PrgC,Carries__1/3,Carries__CPA,Carries__Mis,Carries__Dis,Receiving__Rec,Receiving__PrgR,Performance__CrdY,Performance__CrdR,Performance__2CrdY,Performance__Fls,Performance__Fld,Performance__Off,Performance__Crs,Performance__Int,Performance__TklW,Performance__PKwon,Performance__PKcon,Performance__OG,Performance__Recov,Aerial Duels__Won,Aerial Duels__Lost,Aerial Duels__Won%,Season,Team
0,2017-08-18 20:15:00,La Liga,1,5,0,0,0,1,Leganés,1.1,1.3,47.0,9231.0,Manu García,4-4-2,4-2-3-1,José Munuera,0,8.0,4.0,50.0,0.00,0.00,22.8,0.0,0,1,1.1,0.4,0.04,-1.1,-0.4,3.0,1.0,2.0,66.7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,21.0,23.8,30.0,4.0,46.7,33.0,7.0,100.0,57.3,11.0,1.0,9.1,2.0,14.3,252.0,385.0,65.5,4587.0,1900.0,119.0,145.0,82.1,98.0,138.0,71.0,33.0,82.0,40.2,0,0.3,0.4,7.0,19.0,2.0,0.0,18.0,385.0,330.0,55.0,19.0,1.0,5.0,10.0,25.0,2.0,1.0,0.0,1.0,252.0,0.0,10.0,15.0,7.0,3.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,13.0,10.0,9.0,0.0,8.0,15.0,53.3,7.0,14.0,5.0,9.0,14.0,33,17.0,1.0,47.0,503.0,47.0,200.0,222.0,89.0,10.0,502.0,18.0,8.0,44.4,10.0,55.6,235.0,1359.0,652.0,10.0,8.0,4.0,19.0,13.0,250.0,18.0,1,0,0,17.0,17.0,0.0,10.0,14.0,13.0,1.0,0.0,0.0,60.0,16.0,30.0,34.8,2017,Alaves
1,2017-08-26 18:15:00,La Liga,2,6,1,0,0,2,Barcelona,0.9,2.5,27.0,19356.0,Manu García,4-5-1,4-3-3,Carlos del Cerro,0,11.0,2.0,18.2,0.00,0.00,21.7,2.0,0,0,0.9,0.9,0.09,-0.9,-0.9,11.0,2.0,8.0,81.8,0.0,2.3,0.3,1.0,0.0,1.0,0.0,10.0,17.0,58.8,26.0,4.0,53.8,38.9,5.0,60.0,48.6,13.0,0.0,0.0,0.0,7.7,202.0,281.0,71.9,3959.0,1914.0,85.0,99.0,85.9,73.0,97.0,75.3,34.0,67.0,50.7,0,0.8,0.3,6.0,14.0,1.0,0.0,13.0,281.0,257.0,23.0,7.0,1.0,4.0,5.0,7.0,1.0,1.0,0.0,0.0,202.0,1.0,4.0,19.0,9.0,2.0,2.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,18.0,19.0,4.0,2.0,11.0,32.0,34.4,21.0,14.0,7.0,7.0,15.0,40,30.0,1.0,27.0,396.0,80.0,213.0,140.0,48.0,9.0,396.0,16.0,8.0,50.0,8.0,50.0,258.0,1290.0,662.0,7.0,2.0,4.0,10.0,5.0,199.0,13.0,2,0,0,12.0,9.0,1.0,5.0,15.0,18.0,0.0,1.0,0.0,57.0,8.0,6.0,57.1,2017,Alaves
2,2017-09-10 18:30:00,La Liga,3,7,0,0,0,1,Celta Vigo,0.5,2.7,34.0,17384.0,Fernando Pacheco,4-1-4-1,4-3-3,Mario Melero,0,5.0,3.0,60.0,0.00,0.00,1

In [8]:
object_columns = df2.select_dtypes(include=['object']).columns
print(object_columns)

Index(['Comp', 'Opponent', 'Captain', 'Formation', 'Opp Formation', 'Referee',
       'Team'],
      dtype='object')
